# 🏦 AL Rajhi Bank Stock Forecasting: XGBoost Analysis with SHAP

## Project Walkthrough

### Project Summary
Built a comprehensive XGBoost-based stock price forecasting model for AL Rajhi Bank (1120.SR) covering all 6 prompts.

---

### What Was Accomplished
| Prompt | Description | Status |
|--------|-------------|--------|
| 1 | Problem Definition & Model Selection | ✅ |
| 2 | Feature Engineering (57 features) | ✅ |
| 3 | Modeling Pipeline with GridSearchCV | ✅ |
| 4 | Explainability & Business Insights | ✅ |
| 5 | Data Storytelling Narrative | ✅ |
| 6 | Dashboard Design Concept | ✅ |

---

In [ ]:
from IPython.display import Image
Image(filename = "AL Rajhi Bank Stock Forecasting XGBoost.png", width = 500, height = 500)

## 📦 Imports & Setup

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# SHAP for Explainability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    print("⚠️ SHAP not installed. Run: %pip install shap")
    SHAP_AVAILABLE = False

# Plotly for Interactive Visualizations
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    print("⚠️ Plotly not installed. Run: %pip install plotly")
    PLOTLY_AVAILABLE = False

print("✅ All libraries imported successfully!")

## 📂 Load Data

In [ ]:
df = pd.read_csv('AL_Rjehei.csv')
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df = df.sort_values('Date').reset_index(drop=True)

print(f"📊 Dataset Shape: {df.shape}")
print(f"📅 Date Range: {df['Date'].min()} to {df['Date'].max()}")
print(f"📈 Total Trading Days: {len(df)}")
df.head()

## ✅ Prompt 1: Problem Definition & Model Selection

### 🎯 Target Variable & Problem Type
- **Target**: `Close` (Closing Price)
- **Problem Type**: **Regression + Time-Series Forecasting**

### 📌 Why XGBoost is Suitable
| Reason | Description |
|--------|-------------|
| Non-linear | Handles complex non-linear relationships |
| Feature Interactions | Captures feature interactions automatically |
| Missing Values | Built-in handling of missing values |
| Regularization | L1/L2 regularization prevents overfitting |
| Interpretability | Feature importance for explainability |
| Speed | Faster than deep learning models |

### ⚠️ Assumptions & Limitations
1. Does NOT natively handle sequential dependencies → Use lag features
2. Assumes patterns persist → May fail during regime changes
3. Sensitive to hyperparameters → Use GridSearchCV
4. Cannot extrapolate beyond training range

## ✅ Prompt 2: Feature Engineering (XGBoost Optimized)

In [ ]:
# Create a copy for feature engineering
df_ml = df.copy()

# 1️⃣ DATE-BASED FEATURES
print("📅 Creating Date-Based Features...")
df_ml['Year'] = df_ml['Date'].dt.year
df_ml['Month'] = df_ml['Date'].dt.month
df_ml['Day'] = df_ml['Date'].dt.day
df_ml['DayOfWeek'] = df_ml['Date'].dt.dayofweek
df_ml['Quarter'] = df_ml['Date'].dt.quarter
df_ml['WeekOfYear'] = df_ml['Date'].dt.isocalendar().week.astype(int)
df_ml['IsMonthStart'] = df_ml['Date'].dt.is_month_start.astype(int)
df_ml['IsMonthEnd'] = df_ml['Date'].dt.is_month_end.astype(int)
print("  ✓ Year, Month, Day, DayOfWeek, Quarter, WeekOfYear")

In [ ]:
# 2️⃣ LAG FEATURES
print("⏳ Creating Lag Features...")
lag_periods = [1, 2, 3, 5, 10, 20]
for lag in lag_periods:
    df_ml[f'Close_Lag_{lag}'] = df_ml['Close'].shift(lag)
    df_ml[f'Volume_Lag_{lag}'] = df_ml['Volume'].shift(lag)
print(f"  ✓ Lag features for periods: {lag_periods}")

In [ ]:
# 3️⃣ ROLLING STATISTICS
print("📊 Creating Rolling Statistics...")
windows = [5, 10, 20, 50]
for window in windows:
    df_ml[f'MA_{window}'] = df_ml['Close'].rolling(window=window).mean()
    df_ml[f'Rolling_Std_{window}'] = df_ml['Close'].rolling(window=window).std()
    df_ml[f'Rolling_Min_{window}'] = df_ml['Close'].rolling(window=window).min()
    df_ml[f'Rolling_Max_{window}'] = df_ml['Close'].rolling(window=window).max()
print(f"  ✓ Rolling stats for windows: {windows}")

In [ ]:
# 4️⃣ PRICE-DERIVED FEATURES
print("💰 Creating Price-Derived Features...")
df_ml['Daily_Return'] = df_ml['Close'].pct_change() * 100
df_ml['Price_Range'] = df_ml['High'] - df_ml['Low']
df_ml['Price_Range_Pct'] = (df_ml['High'] - df_ml['Low']) / df_ml['Close'] * 100
df_ml['Gap'] = df_ml['Open'] - df_ml['Close'].shift(1)
df_ml['Gap_Pct'] = df_ml['Gap'] / df_ml['Close'].shift(1) * 100
df_ml['High_Low_Ratio'] = df_ml['High'] / df_ml['Low']
df_ml['Close_Open_Diff'] = df_ml['Close'] - df_ml['Open']
print("  ✓ Daily_Return, Price_Range, Gap, High_Low_Ratio")

In [ ]:
# 5️⃣ VOLATILITY FEATURES
print("📈 Creating Volatility Features...")
df_ml['Volatility_5'] = df_ml['Daily_Return'].rolling(window=5).std()
df_ml['Volatility_20'] = df_ml['Daily_Return'].rolling(window=20).std()
print("  ✓ 5-day and 20-day volatility")

In [ ]:
# 6️⃣ TECHNICAL INDICATORS
print("🔧 Creating Technical Indicators...")

# RSI (Relative Strength Index)
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df_ml['RSI_14'] = calculate_rsi(df_ml['Close'], 14)

# MACD
df_ml['EMA_12'] = df_ml['Close'].ewm(span=12, adjust=False).mean()
df_ml['EMA_26'] = df_ml['Close'].ewm(span=26, adjust=False).mean()
df_ml['MACD'] = df_ml['EMA_12'] - df_ml['EMA_26']
df_ml['MACD_Signal'] = df_ml['MACD'].ewm(span=9, adjust=False).mean()
df_ml['MACD_Histogram'] = df_ml['MACD'] - df_ml['MACD_Signal']

# Bollinger Bands
df_ml['BB_Middle'] = df_ml['Close'].rolling(window=20).mean()
df_ml['BB_Std'] = df_ml['Close'].rolling(window=20).std()
df_ml['BB_Upper'] = df_ml['BB_Middle'] + (df_ml['BB_Std'] * 2)
df_ml['BB_Lower'] = df_ml['BB_Middle'] - (df_ml['BB_Std'] * 2)
df_ml['BB_Width'] = (df_ml['BB_Upper'] - df_ml['BB_Lower']) / df_ml['BB_Middle']
df_ml['BB_Position'] = (df_ml['Close'] - df_ml['BB_Lower']) / (df_ml['BB_Upper'] - df_ml['BB_Lower'])

print("  ✓ RSI (14-period)")
print("  ✓ MACD, Signal Line, Histogram")
print("  ✓ Bollinger Bands (Width, Position)")

In [ ]:
# 7️⃣ HANDLE MISSING VALUES
print("🔧 Handling Missing Values...")
initial_rows = len(df_ml)
df_ml = df_ml.dropna()
final_rows = len(df_ml)
print(f"  ✓ Dropped {initial_rows - final_rows} rows with NaN values")
print(f"  ✓ Remaining samples: {final_rows}")

# Feature Summary
feature_cols = [col for col in df_ml.columns if col not in ['Date', 'Close']]
print(f"\n📋 Total Features Created: {len(feature_cols)}")

## ✅ Prompt 3: Modeling Pipeline (Professional)

In [ ]:
# Prepare Features and Target
feature_columns = [col for col in df_ml.columns if col not in ['Date', 'Close', 'Open', 'High', 'Low', 'Volume']]
X = df_ml[feature_columns]
y = df_ml['Close']

print(f"📊 Feature Matrix Shape: {X.shape}")
print(f"🎯 Target Variable: Close Price")

# TIME-AWARE TRAIN/TEST SPLIT
print("\n⏱️ Time-Aware Train/Test Split...")
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
dates_test = df_ml['Date'].iloc[split_idx:]

print(f"  ✓ Training samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  ✓ Testing samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# BASELINE MODEL
print("🚀 Training Baseline XGBoost Model...")
baseline_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)
baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)

baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))
baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_r2 = r2_score(y_test, baseline_pred)

print(f"  Baseline RMSE: {baseline_rmse:.4f}")
print(f"  Baseline MAE: {baseline_mae:.4f}")
print(f"  Baseline R²: {baseline_r2:.4f}")

In [ ]:
# HYPERPARAMETER TUNING
print("🔧 Hyperparameter Tuning with GridSearchCV...")
tscv = TimeSeriesSplit(n_splits=5)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(
    xgb_model,
    param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

print("⏳ Running GridSearchCV (this may take a few minutes)...")
grid_search.fit(X_train, y_train)

print(f"\n✅ Best Parameters: {grid_search.best_params_}")
print(f"✅ Best CV Score (RMSE): {-grid_search.best_score_:.4f}")

In [ ]:
# FINAL MODEL EVALUATION
print("📊 Final Model Evaluation...")
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("\n" + "━"*50)
print("📈 FINAL MODEL PERFORMANCE METRICS")
print("━"*50)
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")
print(f"  R² Score: {r2:.4f}")
print(f"  MAPE: {mape:.2f}%")
print("━"*50)

In [ ]:
# FEATURE IMPORTANCE
print("📊 Feature Importance Analysis...")
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n🔝 Top 10 Most Important Features:")
feature_importance.head(10)

## ✅ Prompt 4: Explainability & Insights (SHAP)

In [ ]:
if SHAP_AVAILABLE:
    print("🔍 Generating SHAP Values...")
    
    # Create SHAP explainer
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test)
    
    # Calculate mean absolute SHAP values
    shap_importance = pd.DataFrame({
        'Feature': feature_columns,
        'SHAP_Importance': np.abs(shap_values).mean(axis=0)
    }).sort_values('SHAP_Importance', ascending=False)
    
    print("\n🔝 Top 10 Features by SHAP Importance:")
    display(shap_importance.head(10))
    
    # SHAP Summary Plot
    print("\n📊 Creating SHAP Summary Plot...")
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=True, max_display=15)
    
else:
    print("⚠️ SHAP not available. Using XGBoost feature importance.")
    shap_importance = feature_importance.copy()
    shap_importance.columns = ['Feature', 'SHAP_Importance']

### 💡 Business Insights

**Top Predictive Features:**
1. Rolling_Max_10 / Rolling_Max_20 - Price ceiling indicators
2. MA_5 - Short-term trend direction
3. EMA_12 - Exponential trend signal
4. Rolling_Min_10 - Support level indicator

**Trading Signals:**
- Lag features (previous day prices) are strong predictors
- Moving averages indicate trend direction
- RSI and MACD provide momentum signals
- Volatility features capture market uncertainty

**Actionable Recommendations:**
- Monitor 5-day and 20-day moving averages for trend changes
- RSI above 70 suggests overbought conditions
- RSI below 30 suggests oversold conditions
- High volatility periods require caution

## ✅ Prompt 5: Data Storytelling Narrative

### 🏦 Business Context
AL Rajhi Bank (1120.SR) is one of the largest Islamic banks in the world and a cornerstone of the Saudi Arabian financial sector. Understanding its stock price dynamics is crucial for investors and analysts alike.

### 📊 Key EDA Findings
- **Long-term Growth**: Positive trend from 2010-2025 reflecting Saudi economic growth
- **Volatility Patterns**: Cyclical patterns correlate with market events
- **Trading Volume**: Higher volumes during significant price moves indicate institutional interest

### 🤖 Why XGBoost Was Chosen
1. Excels at capturing non-linear relationships
2. Built-in regularization prevents overfitting
3. Feature importance provides interpretable insights
4. Faster training than deep learning alternatives

### 💡 Actionable Recommendations
1. **Trend Following**: Use 20-day MA crossovers for entry/exit signals
2. **Momentum Trading**: RSI extremes (<30 or >70) suggest reversal opportunities
3. **Risk Management**: Reduce position sizes during high volatility periods
4. **Retraining**: Update model monthly with new data

## ✅ Prompt 6: Dashboard Design & Visualizations

In [ ]:
if PLOTLY_AVAILABLE:
    print("📊 Generating Interactive Dashboard Visualizations...")
    
    # 1. Predictions vs Actuals
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=dates_test, y=y_test,
        mode='lines', name='Actual',
        line=dict(color='#1f77b4', width=2)
    ))
    fig1.add_trace(go.Scatter(
        x=dates_test, y=y_pred,
        mode='lines', name='Predicted',
        line=dict(color='#ff7f0e', width=2, dash='dash')
    ))
    fig1.update_layout(
        title='📈 Predictions vs Actuals',
        xaxis_title='Date',
        yaxis_title='Close Price (SAR)',
        hovermode='x unified',
        template='plotly_white',
        height=500
    )
    fig1.show()

In [ ]:
if PLOTLY_AVAILABLE:
    # 2. Feature Importance Chart
    top_n = 15
    fig2 = px.bar(
        feature_importance.head(top_n),
        x='Importance', y='Feature',
        orientation='h',
        title='📊 Top 15 Feature Importance (XGBoost)',
        color='Importance',
        color_continuous_scale='Blues'
    )
    fig2.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        template='plotly_white',
        height=500
    )
    fig2.show()

In [ ]:
# Matplotlib visualization as backup
plt.figure(figsize=(12, 6))
plt.plot(dates_test.values, y_test.values, label='Actual', color='#1f77b4', linewidth=2)
plt.plot(dates_test.values, y_pred, label='Predicted', color='#ff7f0e', linestyle='--', linewidth=2)
plt.title('Predictions vs Actuals', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Close Price (SAR)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 📋 Project Summary

In [ ]:
print("="*60)
print("📋 PROJECT SUMMARY")
print("="*60)

print(f"""
🏦 PROJECT: AL Rajhi Bank Stock Forecasting
📊 MODEL: XGBoost Regressor with SHAP Explainability
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ COMPLETED TASKS:
  1. Problem Definition: Regression + Forecasting on Close price
  2. Feature Engineering: {len(feature_columns)} features created
  3. Modeling Pipeline: TimeSeriesSplit CV with GridSearchCV tuning
  4. Explainability: Feature importance analysis
  5. Storytelling: Professional business narrative
  6. Dashboard: Interactive visualizations

📈 FINAL MODEL PERFORMANCE:
  • R² Score: {r2:.4f}
  • RMSE: {rmse:.4f}
  • MAE: {mae:.4f}
  • MAPE: {mape:.2f}%

🏷️ KAGGLE PROJECT NAME:
  "AL Rajhi Bank Stock Forecasting: XGBoost Analysis with SHAP"

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

print("\n✅ Analysis Complete!")